In [1]:
%pip install cvlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.utils import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import cv2
import os
import cvlib as cv

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError:
    pass

In [5]:
dwnld_link = "https://github.com/arunponnusamy/cvlib/releases/download/v0.2.0/gender_detection.model"
model_path = get_file("gender_detection.model", dwnld_link,
                     cache_subdir="pre-trained", cache_dir=os.getcwd())



In [6]:
images_folder = "./drive/MyDrive/Screenshots/"

In [14]:
images = []
for movie_folder in os.listdir(images_folder):
  movie_images = []
  for image_filename in os.listdir(os.path.join(images_folder,movie_folder)):
    image = cv2.imread(os.path.join(images_folder,movie_folder,image_filename))
    if image is None:
      print(image_filename)
      pass
    movie_images.append(image)
  images.append((movie_folder,movie_images))

.DS_Store
.DS_Store
.DS_Store


See how many folders are in the list of images. This will be three: one for Inception, one for Interstellar, and one for The Matrix.

In [15]:
len(images)

3

See how many images are in those folders.

In [17]:
print(len(images[0][1]))
print(len(images[1][1]))
print(len(images[2][1]))

1336
1554
1218


Load the model.

In [18]:
# load pre-trained model
model = load_model(model_path)

Create an output folder if one doesn't already exist.

In [20]:
output_dir = "./drive/MyDrive/Screenshots/output"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

Create a list of categories for our data.

In [24]:
MEN = "men"
WOMEN = "women"
MIXED = "mixed"
categories = [MEN, WOMEN, MIXED]

Create folders within our output folders for the different films. And then create folders within those folders for men/women/mixed.

In [25]:
for movie_name, _ in images:
    movie_output_path = os.path.join(output_dir, movie_name)
    if not os.path.exists(movie_output_path):
        os.mkdir(movie_output_path)
    for category in categories:
        category_path = os.path.join(movie_output_path, category)
        if not os.path.exists(category_path):
            os.mkdir(category_path)

In [ ]:
classes = ['man','woman']
counter = 0

In [ ]:
import copy

In [ ]:
for image in copy.deepcopy(images):
  # detect faces in the image
  try:
    face, confidence = cv.detect_face(image)
  except TypeError:
    pass

  if len(face) == 0:
    pass

  genders = []

  # loop through detected faces
  for idx, f in enumerate(face):

    try:
      # get corner points of face rectangle
      (startX, startY) = f[0], f[1]
      (endX, endY) = f[2], f[3]

      # draw rectangle over face
      cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

      # crop the detected face region
      face_crop = np.copy(image[startY:endY,startX:endX])

      # preprocessing for gender detection model
      face_crop = cv2.resize(face_crop, (96,96))
      face_crop = face_crop.astype("float") / 255.0
      face_crop = img_to_array(face_crop)
      face_crop = np.expand_dims(face_crop, axis=0)

      # apply gender detection on face
      conf = model.predict(face_crop)[0]
      # print(conf)
      # print(classes)

      # get label with max accuracy
      idx = np.argmax(conf)
      label = classes[idx]

      genders.append(classes[idx])

      label = "{}: {:.2f}%".format(label, conf[idx] * 100)

      Y = startY - 10 if startY - 10 > 10 else startY + 10

      # write label and confidence above face rectangle
      cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                  0.7, (0, 255, 0), 2)
    except Exception:
      pass

  print(genders)
  pass

  if all([gender == "man" for gender in genders]):
    cv2.imwrite(os.path.join(output_dir, "Inception", "men", str(counter) +".jpg"), image)
  elif all([gender == "woman" for gender in genders]):
    cv2.imwrite(os.path.join(output_dir, "Inception", "woman", str(counter) + ".jpg"), image)
  else:
    cv2.imwrite(os.path.join(output_dir, "Inception", "mixed", str(counter) + ".png"), image)
  counter += 1
  print("saved image")


1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 24ms/step
['man']
saved image
1/1 [==============================] - 0s 28ms/step
['man']
saved image
1/1 [==============================] - 0s 19ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 24ms/step
['man']
saved image
1/1 [==============================] - 0s 26ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
saved image
1/1 [==============================] - 0s 22ms/step
['man']
saved image
1/1 [==============================] - 0s 20ms/step
['man']
saved image
1/1 [==============================] - 0s 19ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
saved image
1/1 [==============================] - 0s 21ms/step
['man']
save

error: ignored

In [ ]:
# display output
cv2.imshow("gender detection", image)

# press any key to close window
cv2.waitKey()

# save output
cv2.imwrite("gender_detection.jpg", image)

# release resources
cv2.destroyAllWindows()